In [ ]:
import numpy as np
import scipy
import sys
import torch
from collections import deque
from functools import reduce

print("===VERSIONS===")
print(f"Python: {sys.version}")
print(f"numpy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Scipy: {scipy.__version__}")

In [ ]:
PARAM_EPSILON = 0
PARAM_ALPHA = 0.02
PARAM_GAMMA = 0.5
PARAM_BETA = 1
PARAM_C = 0.01

In [ ]:
class NormalizedTiling:
    def __init__(self, ndim, ksteps, offset):
        self.ndim = ndim
        self.k = ksteps
        self.offset = offset
        assert self.offset < 0, "Offset must be less than 0"
        assert self.offset >= -1 / self.k, f"Offset cannot be above {-1/self.k}"
        self.counts = np.zeros(np.power(self.k + 1, self.ndim))

    def tile_index(self, state):
        assert len(state) == self.ndim, f"Expected state of dimension {self.ndim}, received {len(state)}"
        shift = (state - self.offset * np.ones(self.ndim)) * self.k
        return sum([int(np.power(self.k + 1, j) * (np.floor(shift[j]) if shift[j] < self.k else self.k)) for j in range(self.ndim)])

class TilingDensity:
    def __init__(self, ndim, ntiles, ksteps):
        self.tiles = [NormalizedTiling(ndim, ksteps, -(i + 1)/(ksteps * ntiles)) for i in range(ntiles)]
        self.total_count = 0

    def count(self, x):
        for tiling in self.tiles:
            idx = tiling.tile_index(x)
            tiling.counts[idx] += 1
        self.total_count += 1

    def density(self, x):
        return sum([tiling.counts[tiling.tile_index(x)] for tiling in self.tiles]) / self.total_count

In [ ]:
import torch.nn.functional as F


class BasicNN(torch.nn.Module):
    def __init__(self, ndim):
        super(BasicNN, self).__init__()
        hls = round(8*ndim / 3) # chosen by vibes
        self.fc1 = torch.nn.Linear(2*ndim, hls)
        self.fc2 = torch.nn.Linear(hls, hls)
        self.fc3 = torch.nn.Linear(hls, 1)

        torch.nn.init.normal_(self.fc1.weight, std=0.1)
        torch.nn.init.normal_(self.fc1.weight, std=0.1)
        torch.nn.init.normal_(self.fc3.weight, std=0.1)

        #torch.nn.init.zeros_(self.fc1.bias)
        #torch.nn.init.zeros_(self.fc2.bias)
        #torch.nn.init.zeros_(self.fc3.bias)

        self.double()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def idenfeatures(states, actions):
    return np.array([*states, *actions])

get_features = idenfeatures

In [ ]:
import time
from scipy.stats import gamma


class LinearGammaCrediter:
    def __init__(self, ndims):
        self._history = []
        # From TAMER
        self.k = 2.0
        self.theta = 0.28
        self.delay = 0.20 # seconds

    def add_index(self, feature_vec):
        self._history.append((feature_vec, time.time()))

    def credit(self):
        # Prune old times
        self._history = [x for x in self._history if time.time() - x[1] < gamma.ppf(0.999, self.k, self.delay, self.theta)]
        self._history.sort(key=lambda x: x[1], reverse=True)
        if len(self._history) == 0:
            raise Exception("Empty history array - cannot assign credit")
        # Calculate from remaining
        return sum([
            (gamma.cdf(x[1], self.k, self.delay, self.theta) - \
             (0 if idx == 0 else gamma.cdf(self._history[idx-1][1], self.k, self.delay, self.theta))) * \
            x[0] for idx, x in enumerate(self._history)
        ])

class GammaCrediter:
    """
    Unlike the above/older version of the crediter, this version
    does not combine the history into one feature vector. Instead,
    each vector is returned alongside a weight. Weights sum to 1.
    """
    def __init__(self, ndims):
        self._history = []
        self.k = 2.0
        self.theta = 0.28
        self.delay = 0.20
        
    def add_index(self, feature_vec):
        self._history.append((feature_vec, time.time()))
    
    def credit(self):
        # Prune old times
        self._history = [x for x in self._history if time.time() - x[1] < gamma.ppf(0.999, self.k, self.delay, self.theta)]
        self._history.sort(key=lambda x: x[1], reverse=True)
        if len(self._history) == 0:
            raise Exception("Empty history array - cannot assign credit")
        current_time = time.time()
        weights = np.array([gamma.cdf(current_time - x[1], self.k, self.delay, self.theta) - \
                  (0 if idx == 0 else gamma.cdf(current_time - self._history[idx-1][1], self.k, self.delay, self.theta)) \
                  for idx, x in enumerate(self._history)]).reshape((len(self._history), 1))
        return (np.array([x[0] for x in self._history]), weights)

class UniformCrediter:
    """
    Equally splits reward over interval of t-0.2 to t-4. Weights sum to 1
    """
    def __init__(self, ndims):
        self._history = []
        self._low = 0.2
        self._high = 4.0

    def add_index(self, feature_vec):
        self._history.append((feature_vec, time.time()))

    def credit(self):
        # Prune old times
        call_time = time.time()
        self._history = [x for x in self._history if call_time - x[1] <= self._high]
        tmp = [x for x in self._history if call_time - x[1] >= self._low]
        if len(tmp) == 0:
            raise Exception("No eligible history - cannot assign credit")
        return (np.array([x[0] for x in tmp]), np.ones((len(tmp), 1)) / len(tmp)) 

    def credit2(self):
        call_time = time.time()
        self._history = [x for x in self._history if call_time - x[1] <= self._high] # Limit to those eligible to receive credit
        values = []
        next = []
        for i in range(len(self._history)):
            if call_time - self._history[i][1] >= self._low:
                values.append(self._history[i][0])
                if i < len(self._history) - 1:
                    next.append(self._history[i + 1][0])
                else:
                    # This case shouldn't happen
                    print("Entire history eligible for credit, this shouldn't happen in practice!")
                    next.append(0) # Dummy value
            else:
                break
        assert len(values) == len(next), f"{len(values)} values eligible, but {len(next)} next state-actions found"
        if len(values) == 0:
            raise Exception("No eligible history - cannot assign credit")
        weights = np.ones((len(values), 1)) / len(values)
        return (np.array(values), weights, np.array(next))
        

In [ ]:
def and_op(x, y):
    return x and y

# Scurto used alpha = 0.002
class NeuralSGDAgent:
    def __init__(self, ndims, step, epsilon=PARAM_EPSILON, alpha=PARAM_ALPHA, crediter=UniformCrediter, gamma=PARAM_GAMMA):
        self._ndims = ndims
        self._step = step
        self.crediter = crediter(self._ndims)
        self.state = np.zeros(self._ndims)
        self.state_lows = np.zeros(self._ndims)
        self.state_highs = np.ones(self._ndims)
        self._net = BasicNN(self._ndims)
        self._criterion = torch.nn.MSELoss()
        self._optimizer = torch.optim.SGD(self._net.parameters(), lr=alpha)
        a = np.eye(self._ndims) * self._step
        self._actions = np.concatenate((a, -a))
        self._exclude_dims = set()
        self._rng = np.random.default_rng()
        self._epsilon = epsilon
        self._alpha = alpha # taken from scurto et al 2021
        self._beta = PARAM_BETA
        self._c = PARAM_C
        self._gamma = gamma
        n_tiles = int(2 + np.ceil(np.log2(self._ndims)))
        k_tile = int(np.ceil(1 / (4 * self._step)))
        #n_tiles = int(2 + np.ceil(np.log2(self._ndims)))
        #k_tile = int(np.ceil(1/(2 * self._step)))
        self.tiling = TilingDensity(self._ndims, n_tiles, k_tile)

    def set_state(self, state, *, lows=None, highs=None, action=None, history=True):
        # Check range and update
        if lows is not None:
            assert len(lows) == self._ndims, f"Expected lows to contain {self._ndims} elements, not {len(lows)}"
            assert reduce(and_op, [x >= 0 for x in lows], True) and reduce(and_op, [x <= 1 for x in lows], True), f"Low range not normalized: {lows}"
            self.state_lows = lows
        if highs is not None:
            assert len(highs) == self._ndims, f"Expected highs to contain {self._ndims} elements, not {len(highs)}"
            assert reduce(and_op, [x >= 0 for x in highs], True) and reduce(and_op, [x <= 1 for x in highs], True), f"High range not normalized: {highs}"
            self.state_highs = highs
        assert reduce(and_op, [x >= 0 for x in state], True) and reduce(and_op, [x <= 1 for x in state], True), f"State out of bounds {state}"
        if history:
            self.tiling.count(self.state)
            if action is not None:
                self.crediter.add_index(get_features(self.state, action))
        self.state = state

    def check_bounds(self, state):
        return ((state >= 0) & (state <= 1) & (state >= np.array(self.state_lows)) & (state <= np.array(self.state_highs))).all(0)

    def get_value(self, state, action):
        return self._net(torch.from_numpy(get_features(state, action))).item()

    def select_action(self):
        max_actions = []
        invs = []
        max_value = np.NINF
        for action in self.included_actions():
            next_state = self.state + action
            if self.check_bounds(next_state):
                reward_value = self.get_value(next_state, action)
                explore_value = self._beta * np.power(self.tiling.density(next_state) * self.tiling.total_count + self._c, -0.5)
                value = reward_value + explore_value
                if np.isclose(max_value, value):
                    max_actions.append(action)
                elif value > max_value:
                    max_value = value
                    max_actions = [action]
            else:
                invs.append(action)
        if len(max_actions) > 0:
            return max_actions[self._rng.integers(len(max_actions))]
        else:
            print("No valid actions!")
            return None

    def select_epsilon_greedy_action(self):
        if self._rng.random() < self._epsilon:   
            # Random action
            invalid = True
            actions = self.included_actions()
            if len(actions) > 0:
                while invalid:
                    action = actions[self._rng.integers(len(actions))]
                    next_state = self.state + action
                    invalid = not self.check_bounds(next_state)
                print(f"Taking random action {action}")
                return action
            else:
                print("No valid actions!")
                return None
        else:
            return self.select_action() 

    def apply_action(self, action):
        next_state = self.state + action
        if self.check_bounds(next_state):
            self.set_state(next_state, action=action)
        else:
            raise Exception(f"Tried to transition to an invalid state {next_state}.")

    def process_guiding_reward(self, reward):
        try:
            credit_x, credit_y, next = self.crediter.credit2()
            print(credit_x.shape)
            credit_y = credit_y * reward # TODO include discounted approximation from next state
            # credit_x := credit_x + gamma * q(snext, anext, weights)
            if self._gamma > 0:
                start = time.time()
                credit_x = credit_x + self._gamma * self._net(torch.from_numpy(next)).detach().numpy()

        except Exception as e:
            print(f"ERROR: {e}")
            print("Not applying reward...")
            return
        print("determined guidance")
        self._optimizer.zero_grad()
        error = self._criterion(self._net(torch.from_numpy(credit_x)), torch.from_numpy(credit_y))
        print(f"Error: {error}")
        error.backward()
        self._optimizer.step()
        print("updated model")

    def process_zone_reward(self, reward):
        # Positive reward - apply towards this point - negative reward - apply away from this point
        # Directions include those disabled so we properly encode the zone here
        SCALE_FACTOR = 1
        ZONE_STEPS = 3 # Arbitrarily chosen, TODO scale to match number of divisions in a dimension. Note that Scurto et al. effectively used 5.
        features = []
        weights = []
        for action in self._actions:
            state = self.state
            # Iterate through steps and apply 1) reward in direction we want to move 2) -reward in direction we do not want to move
            for _step in range(1, ZONE_STEPS + 1):
                tmp = state + action
                if not self.check_bounds(tmp):
                    break
                features.append(get_features(tmp, action))
                weights.append(np.array([-float(SCALE_FACTOR * reward)]))
                features.append(get_features(state, -action))
                weights.append(np.array([float(SCALE_FACTOR * reward)]))
                
                state = tmp
        self._optimizer.zero_grad()
        error = self._criterion(self._net(torch.from_numpy(np.array(features))), torch.from_numpy(np.array(weights)))
        print(f"Error: {error}")
        error.backward()
        self._optimizer.step()
        
    def update_activation(self, dimension, activation):
        if activation:
            self._exclude_dims.discard(dimension)
        else:
            self._exclude_dims.add(dimension)

    def included_actions(self):
        # Set of actions that do not modify the 0-indexed dimensions in self._exclude_dims
        return np.array([act for act in self._actions if reduce(lambda x, y: x and y, [act[dim] == 0 for dim in self._exclude_dims], True)])
        

In [ ]:
def and_op(x, y):
    return x and y

# Scurto used alpha = 0.002
class SplitNeuralSGDAgent:
    def __init__(self, ndims1, ndims2, step, epsilon=PARAM_EPSILON, alpha=PARAM_ALPHA, crediter=UniformCrediter, gamma=PARAM_GAMMA):
        self._ndims = ndims1 + ndims2
        self._split_index = ndims1
        self._step = step
        self.crediter1 = crediter(ndims1)
        self.crediter2 = crediter(ndims2)
        self.state = np.zeros(self._ndims)
        self.state_lows = np.zeros(self._ndims)
        self.state_highs = np.ones(self._ndims)
        self._net1 = BasicNN(ndims1)
        self._net2 = BasicNN(ndims2)
        self._criterion = torch.nn.MSELoss()
        self._optimizer1 = torch.optim.SGD(self._net1.parameters(), lr=alpha)
        self._optimizer2 = torch.optim.SGD(self._net2.parameters(), lr=alpha)
        a = np.eye(self._ndims) * self._step
        self._actions = np.concatenate((a, -a))
        self._exclude_dims = set()
        self._rng = np.random.default_rng()
        self._epsilon = epsilon
        self._alpha = alpha # taken from scurto et al 2021
        self._beta = PARAM_BETA
        self._c = PARAM_C
        self._gamma = gamma
        n_tiles = int(2 + np.ceil(np.log2(self._ndims)))
        k_tile = int(np.ceil(1 / (4 * self._step)))
        #n_tiles = int(2 + np.ceil(np.log2(self._ndims)))
        #k_tile = int(np.ceil(1/(2 * self._step)))
        self.tiling = TilingDensity(self._ndims, n_tiles, k_tile)

    def split(self, value):
        assert len(value) == self._ndims, f"Expected vector of size {self._ndims}, received {len(value)}"
        return (value[:self._split_index], value[self._split_index:])

    def set_state(self, state, *, lows=None, highs=None, action=None, history=True):
        # Check range and update
        if lows is not None:
            assert len(lows) == self._ndims, f"Expected lows to contain {self._ndims} elements, not {len(lows)}"
            assert reduce(and_op, [x >= 0 for x in lows], True) and reduce(and_op, [x <= 1 for x in lows], True), f"Low range not normalized: {lows}"
            self.state_lows = lows
        if highs is not None:
            assert len(highs) == self._ndims, f"Expected highs to contain {self._ndims} elements, not {len(highs)}"
            assert reduce(and_op, [x >= 0 for x in highs], True) and reduce(and_op, [x <= 1 for x in highs], True), f"High range not normalized: {highs}"
            self.state_highs = highs
        assert reduce(and_op, [x >= 0 for x in state], True) and reduce(and_op, [x <= 1 for x in state], True), f"State out of bounds {state}"
        if history:
            self.tiling.count(self.state)
            if action is not None:
                state1, state2 = self.split(self.state)
                action1, action2 = self.split(action)
                self.crediter1.add_index(get_features(state1, action1))
                self.crediter2.add_index(get_features(state2, action2))
        self.state = state

    def check_bounds(self, state):
        return ((state >= 0) & (state <= 1) & (state >= np.array(self.state_lows)) & (state <= np.array(self.state_highs))).all(0)

    def get_value(self, state, action):
        state1, state2 = self.split(state)
        action1, action2 = self.split(action)
        return self._net1(torch.from_numpy(get_features(state1, action1))).item() + self._net2(torch.from_numpy(get_features(state2, action2))).item()

    def select_action(self):
        max_actions = []
        invs = []
        max_value = np.NINF
        for action in self.included_actions():
            next_state = self.state + action
            if self.check_bounds(next_state):
                reward_value = self.get_value(next_state, action)
                explore_value = self._beta * np.power(self.tiling.density(next_state) * self.tiling.total_count + self._c, -0.5)
                value = reward_value + explore_value
                if np.isclose(max_value, value):
                    max_actions.append(action)
                elif value > max_value:
                    max_value = value
                    max_actions = [action]
            else:
                invs.append(action)
        if len(max_actions) > 0:
            return max_actions[self._rng.integers(len(max_actions))]
        else:
            print("No valid actions!")
            return None

    def select_epsilon_greedy_action(self):
        if self._rng.random() < self._epsilon:   
            # Random action
            invalid = True
            actions = self.included_actions()
            if len(actions) > 0:
                while invalid:
                    action = actions[self._rng.integers(len(actions))]
                    next_state = self.state + action
                    invalid = not self.check_bounds(next_state)
                print(f"Taking random action {action}")
                return action
            else:
                print("No valid actions!")
                return None
        else:
            return self.select_action() 

    def apply_action(self, action):
        next_state = self.state + action
        if self.check_bounds(next_state):
            self.set_state(next_state, action=action)
        else:
            raise Exception(f"Tried to transition to an invalid state {next_state}.")

    def process_guiding_reward(self, reward):
        try:
            credit_x1, credit_y1, next1 = self.crediter1.credit2()
            credit_x2, credit_y2, next2 = self.crediter2.credit2()
            credit_y1 = credit_y1 * reward
            credit_y2 = credit_y2 * reward
            # credit_x := credit_x + gamma * q(snext, anext, weights)
            if self._gamma > 0:
                start = time.time()
                credit_x1 = credit_x1 + self._gamma * self._net1(torch.from_numpy(next1)).detach().numpy()
                credit_x2 = credit_x2 + self._gamma * self._net2(torch.from_numpy(next2)).detach().numpy()

        except Exception as e:
            print(f"ERROR: {e}")
            print("Not applying reward...")
            return
        self._optimizer1.zero_grad()
        error = self._criterion(self._net1(torch.from_numpy(credit_x1)), torch.from_numpy(credit_y1))
        print(f"Error: {error}")
        error.backward()
        self._optimizer1.step()
        self._optimizer2.zero_grad()
        error = self._criterion(self._net2(torch.from_numpy(credit_x2)), torch.from_numpy(credit_y2))
        print(f"Error: {error}")
        error.backward()
        self._optimizer2.step()

    def process_zone_reward(self, reward):
        # Positive reward - apply towards this point - negative reward - apply away from this point
        # Directions include those disabled so we properly encode the zone here
        SCALE_FACTOR = 1
        ZONE_STEPS = 3 # Arbitrarily chosen, TODO scale to match number of divisions in a dimension. Note that Scurto et al. effectively used 5.
        features1 = []
        features2 = []
        weights = []
        for action in self._actions:
            state = self.state
            # Iterate through steps and apply 1) reward in direction we want to move 2) -reward in direction we do not want to move
            for _step in range(1, ZONE_STEPS + 1):
                tmp = state + action
                if not self.check_bounds(tmp):
                    break
                tmp1, tmp2 = self.split(tmp)
                state1, state2 = self.split(state)
                action1, action2 = self.split(action)
                features1.append(get_features(tmp1, action1))
                features2.append(get_features(tmp2, action2))
                weights.append(np.array([-float(SCALE_FACTOR * reward)]))
                features1.append(get_features(state1, -action1))
                features2.append(get_features(state2, -action2))
                weights.append(np.array([float(SCALE_FACTOR * reward)]))
                
                state = tmp
        self._optimizer1.zero_grad()
        error = self._criterion(self._net1(torch.from_numpy(np.array(features1))), torch.from_numpy(np.array(weights)))
        print(f"Error: {error}")
        error.backward()
        self._optimizer1.step()
        self._optimizer2.zero_grad()
        error = self._criterion(self._net2(torch.from_numpy(np.array(features2))), torch.from_numpy(np.array(weights)))
        error.backward()
        self._optimizer2.step()
        
    def update_activation(self, dimension, activation):
        if activation:
            self._exclude_dims.discard(dimension)
        else:
            self._exclude_dims.add(dimension)

    def included_actions(self):
        # Set of actions that do not modify the 0-indexed dimensions in self._exclude_dims
        return np.array([act for act in self._actions if reduce(lambda x, y: x and y, [act[dim] == 0 for dim in self._exclude_dims], True)])
        

In [ ]:
from pythonosc.dispatcher import Dispatcher
from pythonosc.osc_server import ThreadingOSCUDPServer
from pythonosc.udp_client import SimpleUDPClient
from threading import Thread
import time

manualMode = True
agents = {}

agentType = "split"
haptic_dims = 6

ip = "127.0.0.1" # localhost
port = 8080
destPort = 8081

client = SimpleUDPClient(ip, destPort)

def default_handler(address, *args):
    print(f"DEFAULT {address}: {args}")

def auto_switch_handler(address, state, *args):
    print(f"Is Manual {state}")
    manualMode = state

def manual_set(address, element, *args):
    state = args[::3]
    low = args[1::3]
    high = args[2::3]
    agents[element].set_state(state=state, lows=low, highs=high, history=True)
    #print(f"{element}: {agents[element].state}")

def manual_update(address, element, *args):
    state = args[::3]
    low = args[1::3]
    high = args[2::3]
    agents[element].set_state(state=state, lows=low, highs=high, history=False)

def step(address, element):
    old_state = agents[element].state
    action = agents[element].select_epsilon_greedy_action()
    if action is not None:
        #print(f"{element}: Taking action {action}")
        agents[element].apply_action(action)
        #print(f"Transitioned from {old_state} to {agent.state}")
        client.send_message("/controller/agentSet", [element, *agents[element].state])
    else:
        print(f"{element}: All actions excluded! Doing nothing.")

def reward(address, element, reward):
    agents[element].process_guiding_reward(reward)
    # print(f"Weights updated from {old_weights} to {agent._weights}")

def zone_reward(address, element, reward):
    # Calculate length N_STEPS away on each axis, store in agent
    agents[element].process_zone_reward(reward)
    
def activate(address, element, dimension, activation):
    print(f"{element}: Setting dimension {dimension} to {activation}")
    agents[element].update_activation(dimension, activation)
    print(f"{agents[element]._exclude_dims}")

def init(address, element, ndims, step):
    if element in agents:
        print(f"Replacing agent {element} with fresh. {ndims} dimensions, initial step {step} (norm)")
    else:
        print(f"New agent {element} with {ndims} dimensions, initial step {step} (norm)")
    #agents[element] = LinearSGDAgent(ndims, step)
    if agentType == "joint":
        agents[element] = NeuralSGDAgent(ndims, step, crediter=UniformCrediter)
    elif agentType == "split":
        agents[element] = SplitNeuralSGDAgent(haptic_dims, ndims - haptic_dims, step, crediter=UniformCrediter)

def delete(address, element):
    if element in agents:
        print(f"Deleting agent {element} ({agents[element]._ndims} dimensions)")
        del agents[element]
    else:
        print(f"No agent with identifier {element}!")

dispatcher = Dispatcher()
dispatcher.set_default_handler(default_handler)
dispatcher.map("/uistate/setAutonomous", auto_switch_handler)
dispatcher.map("/controller/manualSet", manual_set)
dispatcher.map("/controller/updateManual", manual_update)
dispatcher.map("/controller/step", step)
dispatcher.map("/controller/reward", reward)
dispatcher.map("/controller/activate", activate)
dispatcher.map("/controller/init", init)
dispatcher.map("/controller/zone_reward", zone_reward)

ip = "127.0.0.1" # localhost
port = 8080

with ThreadingOSCUDPServer((ip, port), dispatcher) as server:
    def quit_func(address, *args):
        print("Quit!")
        server.shutdown()
        server.server_close()
    dispatcher.map("/quit", quit_func)
    thread = Thread(target=server.serve_forever)
    thread.start()
    thread.join()
print("And we're out!")